In [ ]:
!nvidia-smi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

In [ ]:
config_path = '/root/data/models/keypoints_detection/2019_05_13_22_13_59//config.json' 
checkpoint_path = '/root/data/models/keypoints_detection/2019_05_13_22_13_59/weights/fish_test-6'

In [ ]:
config = json.load(open(config_path))
config

In [ ]:
class FLAGS(object):
    """ """
    """
    General settings
    """
    input_size = tuple(config["input_size"])
    stages = 3
    joints = 8
    model_path = checkpoint_path
    cmap_radius = 21
    batch_size = 1
    keypoints_order = ["TAIL_NOTCH",
                    "ADIPOSE_FIN",
                    "UPPER_LIP",
                    "ANAL_FIN",
                    "PELVIC_FIN",
                    "EYE",
                    "PECTORAL_FIN",
                    "DORSAL_FIN"]
    normalize = True
    heatmap_size = 64
    joint_gaussian_variance = 1.0
    augmentation = None
    crop = False

## LOAD MODEL

In [ ]:
import tensorflow as tf
import importlib
from models.nets import fish_test
from utils import cpm_utils
import numpy as np

In [ ]:
from tensorflow.python import pywrap_tensorflow
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
# print_tensors_in_checkpoint_file(checkpoint_path, "", False,
#                                      all_tensor_names=True)

In [ ]:
# reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
# var_to_shape_map = reader.get_variable_to_shape_map()
# for key in sorted(var_to_shape_map):
#     print("tensor_name: ", key, reader.get_tensor(key).shape)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device = '/gpu:0'
with tf.device(tf_device):
    model = fish_test.CPM_Model(FLAGS.input_size, 2, FLAGS.stages, FLAGS.joints)

In [ ]:
model.input_images

In [ ]:
model.cmap_placeholder

Restore weights

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()

sess.run(tf.global_variables_initializer())
saver.restore(sess, FLAGS.model_path)

In [ ]:
test_center_map = cpm_utils.gaussian_img(FLAGS.input_size[0],
                                         FLAGS.input_size[0], 
                                         FLAGS.input_size[0] / 2,
                                         FLAGS.input_size[0] / 2,
                                         FLAGS.cmap_radius)
test_center_map = np.reshape(test_center_map, [1, FLAGS.input_size[0], FLAGS.input_size[0], 1])

Data generator

In [ ]:
import glob
import json
import random
import matplotlib.pyplot as plt
import cv2
from utils.utils import DataGenerator, load_image_keypoints

In [ ]:
annotations = glob.glob("/root/data/bati/labels/labels_*")
# annotations = ['/root/data/gtsf_phase_I/2019-05-02/2019-05-02_cogito_annotations.json']
print("Total number of gtsf sessions: {}".format(len(annotations)))

In [ ]:
random.seed(199)

In [ ]:
cutoff = int(len(annotations)*0.8)
random.shuffle(annotations)
train_files = annotations[:cutoff]
val_files = annotations[cutoff:]
print("Number of train files: {}".format(len(train_files)))
print("Number of validation files: {}".format(len(val_files)))

In [ ]:
val_files

In [ ]:
val_annotations = []
for jpath in val_files:
    val_annotations += json.load(open(jpath))
print("Number of validation data: {}".format(len(val_annotations)))

Froward pass loop

In [ ]:
from tqdm import tqdm

In [ ]:
low_rez = {}
high_rez = {}

for ann in tqdm(val_annotations):
    ann["predictions"] = []
    img_input, kp_resized = load_image_keypoints(ann, FLAGS)

    img_input  = img_input / 255.0 - 0.5
    img_input = img_input[np.newaxis, ...]
    image, keypoints = load_image_keypoints(ann, FLAGS, reshape=False)
    height, width, _ = image.shape
    
#     xs = keypoints[:, 0]
#     min_x = np.max([np.min(xs) - 100, 0])
#     ys = keypoints[:, 1]
#     min_y = np.max([np.min(ys) - 100, 0])
    
    with tf.device(tf_device):   
        predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                  model.stage_heatmap,
                                                  ],
                                                 feed_dict={'input_placeholder:0': img_input,
                                                            'cmap_placeholder:0': test_center_map})
    final_stage_heatmap = predict_heatmap.squeeze()  
    todelete = []
#     # manhattan distance
#     for c in range(8):
#         hm = cv2.resize(final_stage_heatmap[..., c], FLAGS.input_size)
#         hm_max = np.where(hm == hm.max())
#         pred_kp = np.array([hm_max[1][0], hm_max[0][0]])
#         todelete.append(pred_kp)
#         gt_kp = np.array(kp_resized[c, :], dtype=np.uint32)
#         man_dist = np.sum(np.abs(pred_kp - gt_kp))
#         if FLAGS.keypoints_order[c] not in low_rez:
#             low_rez[FLAGS.keypoints_order[c]] = []
#         low_rez[FLAGS.keypoints_order[c]].append(man_dist)
#         ann["predictions"].append(pred_kp.tolist())
#         # print("Manhattan distance between pred and gt {} for {}".format(man_dist, FLAGS.keypoints_order[c]))

    #  # manhattan distance
    for c in range(8):
        hm = cv2.resize(final_stage_heatmap[..., c], (width, height))
        hm_max = np.where(hm == hm.max())
        pred_kp = np.array([hm_max[1][0], hm_max[0][0]])
        # pred_kp += np.array([min_x, min_y])
        gt_kp = np.array(keypoints[c, :], dtype=np.uint32)
        man_dist = np.sum(np.abs(pred_kp - gt_kp))
        if FLAGS.keypoints_order[c] not in high_rez:
            high_rez[FLAGS.keypoints_order[c]] = []
        high_rez[FLAGS.keypoints_order[c]].append(man_dist)
        ann["predictions"].append(pred_kp.tolist())
    
    
#     full = cv2.imread(ann['local_path'])
#     predictions = np.array(ann["predictions"])
#     plt.imshow(full)
#     plt.scatter(predictions[:, 0], predictions[:, 1], c='b')
#     plt.scatter(keypoints[:, 0], keypoints[:, 1], c='r', alpha=0.5)
#     plt.show()
#     break
        # print("Manhattan distance between pred and gt {} for {}".format(man_dist, FLAGS.keypoints_order[c]))


Calculate metrics

In [ ]:
distances = {k: [] for k in FLAGS.keypoints_order}
for ann in tqdm(val_annotations):
    if 'predictions' not in ann:
        continue
    if len(ann['predictions']) == 0:
        continue
    
    keypoints = []
    for kp_name in FLAGS.keypoints_order:
        value = ann["Label"][kp_name]
        keypoints.append([int(value[0]["geometry"]["x"]), 
                          int(value[0]["geometry"]["y"])])
    keypoints = np.array(keypoints)
    predictions = np.array(ann['predictions'])
    
    for i, k in enumerate(FLAGS.keypoints_order):
        man_dist = np.sum(np.abs(keypoints[i, :] - predictions[i, :]))
        distances[k].append(man_dist)

In [ ]:
for k,v in distances.items():
    print('Average error for {} is {}'.format(k, np.mean(v)))

In [ ]:
print('Total average man dist: {}'.format(np.array(list(distances.values())).mean()))

In [ ]:
ann = np.random.choice(val_annotations)
full = cv2.imread(ann['local_path'])
img_input, kp_resized = load_image_keypoints(ann, FLAGS)
predictions = np.array(ann["predictions"])
plt.figure(figsize=(15, 15))
plt.imshow(img_input)
plt.scatter(predictions[:, 0], predictions[:, 1])
plt.show()

In [ ]:
with open('/root/thomas/biomass_kp_predictions_val.json', 'w') as f:
    json.dump(val_annotations, f)

Display

In [ ]:
for (k,v) in low_rez.items():
    print("Average error on {}x{} images for keypoint {} is {}".format(FLAGS.input_size[0],
                                                                    FLAGS.input_size[0],k, np.median(v)))

In [ ]:
for (k,v) in high_rez.items():
    print("Average error on full rez images for keypoint {} is {}".format( k, np.median(v)))

# PLOT

In [ ]:
debug = '190313010007'
debug = '190301010006'
debug = '190313010009'
debug = '190301010014'
debug = '190313010011'
debug = '190313010009'
# bug = '190313010008'

debug_ann = [ann for ann in val_annotations if ann["timestamp"] == debug]

In [ ]:
image, keypoints = load_image_keypoints(debug_ann[0], FLAGS, reshape=False)
preds = np.array(debug_ann[0]["predictions"])
f, ax = plt.subplots(1, figsize=(20,10))
ax.imshow(image)
ax.scatter(keypoints[:, 0], keypoints[:, 1], c="r")
ax.scatter(preds[:, 0], preds[:, 1], c="y")
for (i, t) in enumerate(FLAGS.keypoints_order):
    ax.text(preds[i, 0], preds[i, 1], t, {"color": "w"})
plt.show()

In [ ]:
image, keypoints = load_image_keypoints(debug_ann[1], FLAGS, reshape=False)
preds = np.array(debug_ann[1]["predictions"])
f, ax = plt.subplots(1, figsize=(20,10))
ax.imshow(image)
ax.scatter(keypoints[:, 0], keypoints[:, 1], c="r")
ax.scatter(preds[:, 0], preds[:, 1], c="y")
for (i, t) in enumerate(FLAGS.keypoints_order):
    ax.text(preds[i, 0], preds[i, 1], t, {"color": "w"})
plt.show()

# Disparity 

In [ ]:
from utils.utils import load_ann_keypoints

In [ ]:
# create pairs per timestamp
pairs = {}
for ann in val_annotations:
    if ann["species"] != "salmon":
        continue
    if ann["kfactor"] < 0.3:
        continue
    timestamp = ann["timestamp"]
    side = os.path.basename(ann["local_path"]).split("_")[0]
    ann["side"] = side
    if timestamp not in pairs:
        pairs[timestamp] = {}
    pairs[timestamp][side] = ann

full_pairs = [k for (k, v)in pairs.items() if "left" in v and "right" in v]
print("Number of full pairs: {}".format(len(full_pairs)))

In [ ]:
buffer = 100

In [ ]:
ground_truth_disparities = []
pred_disparities= []

for ts in full_pairs:
    left_ann = pairs[ts]["left"]
    right_ann = pairs[ts]["right"]
    
    left_keypoints = load_ann_keypoints(left_ann, FLAGS.keypoints_order)
    xmin = left_keypoints[:, 1].min()
    ymin = left_keypoints[:, 0].min()
    left_predictions = np.array(left_ann["predictions"])
    left_predictions[:, 0] += ymin-buffer
    left_predictions[:, 1] += xmin-buffer
    
    right_keypoints = load_ann_keypoints(right_ann, FLAGS.keypoints_order)
    xmin = right_keypoints[:, 1].min()
    ymin = right_keypoints[:, 0].min()
    right_predictions = np.array(right_ann["predictions"])
    right_predictions[:, 0] += ymin-buffer
    right_predictions[:, 1] += xmin-buffer
    
    ground_truth_disparities.append(left_keypoints[:, 0] - right_keypoints[:, 0])
    pred_disparities.append(left_predictions[:, 0] - right_predictions[:, 0])
    
ground_truth_disparities = np.array(ground_truth_disparities)
pred_disparities = np.array(pred_disparities)

In [ ]:
full_pairs[37]

In [ ]:
mean_error = np.mean(ground_truth_disparities - pred_disparities)
print("Mean disparity error: {}".format(mean_error))
relative_error = np.mean((ground_truth_disparities - pred_disparities)*100 / ground_truth_disparities)
print("Mean relative disparity error: {}".format(relative_error))
abs_relative_error = np.mean(np.abs((ground_truth_disparities - pred_disparities)*100 / ground_truth_disparities))
print("Mean absolute relative disparity error: {}".format(abs_relative_error))

In [ ]:
error = ground_truth_disparities - pred_disparities
np.where(error > 100)

In [ ]:
full_pairs[33]